In [18]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, preprocessing
import pandas as pd
from matplotlib import style
import statistics
style.use("ggplot")

In [2]:
FEATURES =  ['DE Ratio',
             'Trailing P/E',
             'Price/Sales',
             'Price/Book',
             'Profit Margin',
             'Operating Margin',
             'Return on Assets',
             'Return on Equity',
             'Revenue Per Share',
             'Market Cap',
             'Enterprise Value',
             'Forward P/E',
             'PEG Ratio',
             'Enterprise Value/Revenue',
             'Enterprise Value/EBITDA',
             'Revenue',
             'Gross Profit',
             'EBITDA',
             'Net Income Avl to Common ',
             'Diluted EPS',
             'Earnings Growth',
             'Revenue Growth',
             'Total Cash',
             'Total Cash Per Share',
             'Total Debt',
             'Current Ratio',
             'Book Value Per Share',
             'Cash Flow',
             'Beta',
             'Held by Insiders',
             'Held by Institutions',
             'Shares Short (as of',
             'Short Ratio',
             'Short % of Float',
             'Shares Short (prior ']

In [19]:
def Build_Data_Set():
    #data_df = pd.read_csv("key_stats_acc_perf_WITH_NA.csv")
    data_df = pd.read_csv("key_stats_acc_perf_NO_NA.csv")
    #data_df = data_df[:100]
    data_df = data_df.reindex(np.random.permutation(data_df.index))
    data_df = data_df.replace("NaN", 0).replace("N/A", 0)
    
    X = np.array(data_df[FEATURES].values)#.tolist())
    y = (data_df["Status"]
         .replace("underperform",0)
         .replace("outperform",1)
         .values.tolist())

    X = preprocessing.scale(X)

    Z = np.array(data_df[["stock_p_change","sp500_p_change"]])
    return X, y, Z

In [40]:
def Analysis():
    
    test_size = 1000
    invest_amount = 10000
    total_invests = 0
    if_market = 0
    if_strat = 0
    
    X, y, Z = Build_Data_Set()
    print (len (X))
    #Z = np.nan_to_num(X)
    clf =svm.SVC(kernel="linear", C= 1.0)
    clf.fit(X[ :- test_size], y[ :- test_size])
    
    correct_count = 0
    
    for x in range(1, test_size+1) :
        if clf.predict([X[-x]]) [0] == y[-x]:
            correct_count += 1
        if clf.predict([X[-x]]) [0] == 1:
            if np.isnan(Z[-x][0]) or np.isnan(Z[-x][1]):
                continue

            invest_return = invest_amount + (invest_amount * (Z[-x][0]/100))
            market_return = invest_amount + (invest_amount * (Z[-x][1]/100))

            total_invests += 1
            
            if_market += market_return
            if_strat += invest_return

    print("Accuracy:", (correct_count/test_size)* 100.00)
    
    print("Total Trades:", total_invests)
    print("Ending with Strategy", if_strat)
    print("Ending with Market", if_market)

    compared = ((if_strat - if_market) / if_market) * 100.0
    do_nothing = total_invests * invest_amount

    avg_market = ((if_market - do_nothing) / do_nothing) * 100.0
    avg_strat = ((if_strat - do_nothing) / do_nothing) * 100.0

    
    
    print("Compared to market, we earn", str(compared)+"% more")
    print("Average investment return:", str(avg_strat)+"%")
    print("Average market return:", str(avg_market)+"%")
    

Analysis()

C:\Users\riadb\AppData\Local\Temp\ipykernel_16408\3182322414.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace("outperform",1)
C:\Users\riadb\miniconda3\envs\pydata-book\Lib\site-packages\sklearn\preprocessing\_data.py:261: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(


3012
Accuracy: 58.4
Total Trades: 731
Ending with Strategy 8867893.0
Ending with Market 8225308.0
Compared to market, we earn 7.8122910412594875% more
Average investment return: 21.311805745554036%
Average market return: 12.521313269493845%
